In [1]:
import ortools
from ortools.sat.python import cp_model

In [2]:
# Iniciando el CSP model
model = cp_model.CpModel()

# Iniciando el espacio de juego
game_space = []

game_size = 3

for i in range(game_size):
    new_row = []
    for j in range(game_size):
        new_var = model.NewIntVar(1,9,f"Tile [{i},{j}]")
        new_row.append(new_var)
    game_space.append(new_row)

In [3]:
# Aca iniciaremos el estado inicial sin numeros del juego
# Hardcodeado por ahora   
# 
# RULESET 10X10      
# ruleset = [ 
#     [ [[2,1]], 5, "+" ],
#     [ [[0,3]], 5, "+" ],
#     [ [[7,0]], 4, "+" ],
#     [ [[8,0]], 2, "+" ],
#     [ [[2,2]], 2, "+" ],
#     [ [[9,3]], 10, "+" ],
#     [ [[0,4]], 10, "+" ],
#     [ [[0,5]], 6, "+" ],
#     [ [[4,5]], 4, "+" ],
#     [ [[7,5]], 1, "+" ],
#     [ [[8,5]], 8, "+" ],
#     [ [[0,6]], 8, "+" ],
#     [ [[3,6]], 1, "+" ],
#     [ [[4,6]], 10, "+" ],
#     [ [[0,7]], 5, "+" ],
#     [ [[2,7]], 1, "+" ],
#     [ [[5,7]], 10, "+" ],
#     [ [[9,7]], 9, "+" ],
#     [ [[1,9]], 9, "+" ],
#     [ [[3,9]], 4, "+" ],
#     [ [[6,9]], 4, "+" ],
#     [ [[0,0],[0,1]], 7, "+" ],
#     [ [[4,0],[5,0]], 63, "*" ],
#     [ [[3,1],[4,1]], 56, "*" ],
#     [ [[7,1],[8,1]], 13, "+" ],
#     [ [[9,0],[9,1]], 7, "+" ],
#     [ [[3,3],[4,3]], 12, "+" ],
#     [ [[5,3],[6,3]], 6, "+" ],
#     [ [[2,4],[3,4]], 7, "+" ],
#     [ [[4,4],[5,4]], 6, "*" ],
#     [ [[6,4],[7,4]], 10, "*" ],
#     [ [[3,7],[4,7]], 12, "*" ],
#     [ [[6,6],[6,7]], 16, "+" ],
#     [ [[7,6],[7,7]], 40, "*" ],
#     [ [[0,8],[0,9]], 11, "+" ],
#     [ [[3,8],[4,8]], 10, "*" ],
#     [ [[5,8],[6,8]], 12, "+" ],
#     [ [[4,9],[5,9]], 7, "*" ],
#     [ [[1,0],[2,0],[1,1]], 20, "+" ],
#     [ [[6,0],[5,1],[6,1]], 16, "+" ],
#     [ [[0,2],[1,2],[0,3]], 14, "+" ],
#     [ [[2,2],[4,2],[5,2]], 400, "*" ],
#     [ [[6,2],[7,2],[7,3]], 162, "*" ],
#     [ [[8,2],[9,2],[8,3]], 10, "+" ],
#     [ [[1,3],[2,3],[1,4]], 22, "+" ],
#     [ [[8,4],[9,4],[9,5]], 19, "+" ],
#     [ [[1,5],[2,5],[3,5]], 21, "+" ],
#     [ [[5,5],[6,5],[5,6]], 15, "+" ],
#     [ [[1,6],[2,6],[1,7]], 13, "+" ],
#     [ [[8,6],[9,6],[8,7]], 84, "*" ],
#     [ [[1,8],[2,8],[2,9]], 14, "+" ],
#     [ [[7,8],[8,8],[7,9]], 420, "*" ],
#     [ [[8,9],[9,8],[9,9]], 15, "+" ]
# ]

ruleset = [
    [ [[0,0],[1,0],[2,0]], 6, "+" ],
    [ [[0,1]], 3, "+" ],
    [ [[0,2],[1,1],[1,2]], 6, "*" ],
    [ [[2,1]], 2, "+" ],
    [ [[2,2]], 1, "+"]
]

In [6]:
def get_NIVs_of_space(space):
    newintvars = []
    # Cada tile en el espacio del constraint...
    for tile in space:
        # ...tiene su correspondiente NIV, asi que lo añadimos a una lista temporal
        newintvars.append(game_space[tile[0]][tile[1]])

    return newintvars

def generate_sum_constraint(space,result):
    NIVs = get_NIVs_of_space(space)
    # La suma de todos los NIVs de este espacio deben igualar al resultado
    model.Add(sum(NIVs) == result)

def generate_mult_constraint(space,result):
    NIVs = get_NIVs_of_space(space)

    # Tristemente ortools no soporta multiplicaciones grandes, AddMultiplicationEquality
    # solo funciona en el estilo a = b * c
    #
    # Asi que vamos a hacer una cadena de multiplicaciones
    #
    # Digamos que tenemos:
    #
    #   x1 * x2 * x3 * x4 = R
    #
    # Tonces hariamos:
    #
    #   1 * x1 * x2 * x3 * x4 = R
    #   x1 * x2 * x3 * x4 = R
    #   x12 * x3 * x4 = R
    #   x123 * x4 = R
    #   
    # y al final tenemos:
    #
    #   x1234 = R

    # Iniciamos la multiplicacion en 1
    product = model.NewConstant(1)
    for niv in NIVs:
        # NIV para representar el nuevo producto
        new_product = model.NewIntVar(1,result,"")

        # Ahora le decimos al modelo que 
        # x1 * x2 = x12
        model.AddMultiplicationEquality(new_product, [product,niv])

        # Actualizamos el producto
        product = new_product
    
    model.Add(product == result)

def generate_sub_constraint(space,result):
    NIVs = get_NIVs_of_space(space)
    # Al contratio de la suma y la multiplicacion, el orden de los factores si importa!
    #
    #   a - b != b - a
    #   a / b != b / a
    #
    # Es por esto que en KenKen, las celdas de sub y div son a lo máximo de tamaño 2
    # Si encontramos que a-b OR b-a == resultado, entonces si se puede solucionar la celda (misma idea para la division)

    # Operadores de nuestra resta
    a = NIVs[0]
    b = NIVs[1]

    # Estas variables verifican si es que alguna de las dos maneras de restar cumplen
    way1 = model.NewBoolVar("")
    way2 = model.NewBoolVar("")

    # (a-b == result) si (way1 == true)
    model.Add(a-b == result).OnlyEnforceIf(way1)
    # (b-a == result) si (way2 == true)
    model.Add(b-a == result).OnlyEnforceIf(way2)

    # Con esto, el modelo va a chequear si es que o way1 o way2 es resultado
    # y por lo tanto, eligira el primero que cumpla verdadero
    model.AddBoolOr([way1,way2])

def generate_div_constraint(space,result):
    NIVs = get_NIVs_of_space(space)

    # Operadores de nuestra division
    a = NIVs[0]
    b = NIVs[1]

    # Estas variables verifican si es que alguna de las dos maneras de dividir cumplen
    way1 = model.NewBoolVar("")
    way2 = model.NewBoolVar("")

    # (a/b == result) si (way1 == true)
    model.Add(a/b == result).OnlyEnforceIf(way1)
    # (b/a == result) si (way2 == true)
    model.Add(b/a == result).OnlyEnforceIf(way2)

    # Con esto, el modelo va a chequear si es que o way1 o way2 es resultado
    # y por lo tanto, eligira el primero que cumpla verdadero
    model.AddBoolOr([way1,way2])

def generate_exp_constraint(space,result):
    NIVs = get_NIVs_of_space(space)

    a = NIVs[0]
    b = NIVs[1]

    way1 = model.NewBoolVar("")
    way2 = model.NewBoolVar("")


    model.Add(a**b == result).OnlyEnforceIf(way1)
    model.Add(b**a == result).OnlyEnforceIf(way2)

def generate_min_constraint(space,result):
    NIVs = get_NIVs_of_space(space)

    for niv in NIVs:
        model.add(niv >= result)

def generate_max_constraint(space,result):
    NIVs = get_NIVs_of_space(space)

    for niv in NIVs:
        model.add(niv <= result)

def generate_mean_constraint(space,result):
    NIVs = get_NIVs_of_space(space)
    N = len(NIVs)
    sum_var = model.NewIntVar(N * 1, N * 9, "")
    model.Add(sum_var == sum(NIVs))

    required_sum = result * N
    if required_sum != int(required_sum):
        raise ValueError("No possible integer solution: target*N is not integer.")

    model.Add(sum_var == int(required_sum))

def generate_mod_constraint(space, result):
    NIVs = get_NIVs_of_space(space)
    a, b = NIVs[0], NIVs[1]

    way1 = model.NewBoolVar("")
    way2 = model.NewBoolVar("")

    # a % b = result   → a = b*q + result
    q1 = model.NewIntVar(0, 9, "")
    model.Add(a == b * q1 + result).OnlyEnforceIf(way1)

    # b % a = result
    q2 = model.NewIntVar(0, 9, "")
    model.Add(b == a * q2 + result).OnlyEnforceIf(way2)

    model.AddBoolOr([way1, way2])

def generate_range_constraint(space, result):
    NIVs = get_NIVs_of_space(space)

    max_v = model.NewIntVar(1, 9, "")
    min_v = model.NewIntVar(1, 9, "")

    model.AddMaxEquality(max_v, NIVs)
    model.AddMinEquality(min_v, NIVs)

    model.Add(max_v - min_v == result)

def generate_pair_product_max_constraint(space, result):
    NIVs = get_NIVs_of_space(space)
    pair_products = []

    for i in range(len(NIVs)):
        for j in range(i+1, len(NIVs)):
            p = model.NewIntVar(1, 81, "")
            model.AddMultiplicationEquality(p, [NIVs[i], NIVs[j]])
            pair_products.append(p)

    max_p = model.NewIntVar(1, 81, "")
    model.AddMaxEquality(max_p, pair_products)

    model.Add(max_p == result)

def generate_sum_squares_constraint(space, result):
    NIVs = get_NIVs_of_space(space)

    squares = []
    for niv in NIVs:
        sq = model.NewIntVar(1, 81, "")
        model.AddMultiplicationEquality(sq, [niv, niv])
        squares.append(sq)

    model.Add(sum(squares) == result)

In [ ]:
# Constraints de las filas y columnas

for i in range(game_size):
    model.AddAllDifferent(game_space[i])

    column = [game_space[j][i] for j in range(game_size)]
    model.AddAllDifferent(column)

# Leyendo cada regla, creando un cosntraint por regla
for rule in ruleset:

    rule_space = rule[0]
    rule_result = rule[1]
    rule_opp = rule[2]

    # Si es que solo hay una tile en el espacio
    # Tonces igualarlo al resultado, de ahi saltar
    if (len(rule_space) == 1):
        NIVs = get_NIVs_of_space(rule_space)[0]
        model.Add(NIVs == rule_result)
        continue


    match rule_opp:
        case "+":
            # COnstraint the suma y all different
            generate_sum_constraint(rule_space,rule_result)
            pass
        case "*":
            # Constraint de mult y all different
            generate_mult_constraint(rule_space,rule_result)
            pass
        case "/":
            # Constraint the div y all different
            generate_sub_constraint(rule_space,rule_result)
            pass
        case "-":
            # Constraint the resta y all different
            generate_div_constraint(rule_space,rule_result)
            pass
        case "^":
            generate_exp_constraint(rule_space,rule_result)
            pass
        case "min":
            generate_min_constraint(rule_space,rule_result)
            pass
        case "max":
            generate_max_constraint(rule_space,rule_result)
            pass
        case "mean":
            generate_mean_constraint(rule_space,rule_result)
            pass
        case "%":
            generate_mod_constraint(rule_space, rule_result)
        case "range":
            generate_range_constraint(rule_space, rule_result)
        case "pairprod":
            generate_pair_product_max_constraint(rule_space, rule_result)
        case "sumsq":
            generate_sum_squares_constraint(rule_space, rule_result)
        case _:
            # Constraint de suma y all different como default
            generate_sum_constraint(rule_space,rule_result)
            pass

In [6]:
for i, ct in enumerate(model.Proto().constraints):
    print(f"Constraint {i}: {ct}")


Constraint 0: all_diff {
  exprs {
    vars: 0
    coeffs: 1
  }
  exprs {
    vars: 1
    coeffs: 1
  }
  exprs {
    vars: 2
    coeffs: 1
  }
}

Constraint 1: all_diff {
  exprs {
    vars: 0
    coeffs: 1
  }
  exprs {
    vars: 3
    coeffs: 1
  }
  exprs {
    vars: 6
    coeffs: 1
  }
}

Constraint 2: all_diff {
  exprs {
    vars: 3
    coeffs: 1
  }
  exprs {
    vars: 4
    coeffs: 1
  }
  exprs {
    vars: 5
    coeffs: 1
  }
}

Constraint 3: all_diff {
  exprs {
    vars: 1
    coeffs: 1
  }
  exprs {
    vars: 4
    coeffs: 1
  }
  exprs {
    vars: 7
    coeffs: 1
  }
}

Constraint 4: all_diff {
  exprs {
    vars: 6
    coeffs: 1
  }
  exprs {
    vars: 7
    coeffs: 1
  }
  exprs {
    vars: 8
    coeffs: 1
  }
}

Constraint 5: all_diff {
  exprs {
    vars: 2
    coeffs: 1
  }
  exprs {
    vars: 5
    coeffs: 1
  }
  exprs {
    vars: 8
    coeffs: 1
  }
}

Constraint 6: linear {
  vars: 0
  vars: 3
  vars: 6
  coeffs: 1
  coeffs: 1
  coeffs: 1
  domain: 6
  domain: 6

In [7]:
for i, v in enumerate(model.Proto().variables):
    print(f"Var {i}: domain={v.domain}, name={v.name}")


Var 0: domain=[1, 9], name=Tile [0,0]
Var 1: domain=[1, 9], name=Tile [0,1]
Var 2: domain=[1, 9], name=Tile [0,2]
Var 3: domain=[1, 9], name=Tile [1,0]
Var 4: domain=[1, 9], name=Tile [1,1]
Var 5: domain=[1, 9], name=Tile [1,2]
Var 6: domain=[1, 9], name=Tile [2,0]
Var 7: domain=[1, 9], name=Tile [2,1]
Var 8: domain=[1, 9], name=Tile [2,2]
Var 9: domain=[1, 1], name=
Var 10: domain=[1, 6], name=
Var 11: domain=[1, 6], name=
Var 12: domain=[1, 6], name=


In [8]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    for i in range(game_size):
        row = []
        for j in range(game_size):
            row.append(str(solver.Value(game_space[i][j])))
        print(" ".join(row))
else:
    print("No solution found")

1 3 2
2 1 3
3 2 1
